# Circuit Precompiling
---
### Basic Information
**Description:** This script conducts the circuit precompilation using `qiskit.transpile`.

In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit.transpiler import CouplingMap

cmap_default = CouplingMap().from_grid(5, 5)

def precompile(circuit, cmap=cmap_default, seed=1000):
    # this function uses the default circuit compilation function of qiskit for precompilation

    # precompile the circuit
    compiled_circuit = transpile(circuit, coupling_map=cmap, seed_transpiler=seed)

    # remove the deactivated qubits from the compiled circuit
    file = open("data/reduced_circuit.qasm", "w")
    file.truncate(0) # make the file empty
    qasm_string = ""

    # add head for the qasm file
    qasm_string += 'OPENQASM 2.0;\n'
    qasm_string += 'include "qelib1.inc";\n'
    qasm_string += f'qreg q[{circuit.num_qubits}];\n'

    for item in compiled_circuit.data:
        gate_type = item[0].name
        gate_size = len(item[1])
        logical_qubit_indices = []
        for qubit in item[1]:
            physical_qubit_index = qubit.index
            logical_qubit_index = compiled_circuit.layout.initial_layout[physical_qubit_index].index
            logical_qubit_indices.append(logical_qubit_index)
        if gate_size == 1:
            if len(item[0].params) == 3:
                params = f'({item[0].params[0]}, {item[0].params[1]}, {item[0].params[2]})'
            elif any(item[0].params) == False:
                params = ''
            else:
                message = 'Unsupported single-qubit gate exists. This code only supports gates without parameters and single-qubit rotation.'
                raise ValueError(message)

            qasm_string += f'{gate_type}{params} q[{logical_qubit_indices[0]}];\n'
        if gate_size == 2:
            qasm_string += f'{gate_type} q[{logical_qubit_indices[0]}], q[{logical_qubit_indices[1]}];\n'

    file.write(qasm_string)
    file.close()

    reduced_circuit = QuantumCircuit.from_qasm_file('data/reduced_circuit.qasm')

    return reduced_circuit